In [1]:
import keras
from keras.models import Sequential, Model, model_from_json
from keras.layers import LSTM, Dense, Dropout, Input, Flatten, concatenate, Reshape
from keras.optimizers import RMSprop, SGD
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, History, Callback, LambdaCallback
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os
import time
from matplotlib import pyplot
import glob
import csv
import re
import tensorflow as tf

Using TensorFlow backend.


In [11]:
def ShuffleData(x_order_addr, y_order_addr):
    x_order = np.load(x_order_addr)
    y_order = np.load(y_order_addr)
    idx_shuffle = np.array(range(x_order.shape[0]))
    np.random.shuffle(idx_shuffle)
    x_shuffle = x_order[idx_shuffle]
    y_shuffle = y_order[idx_shuffle]
    return x_shuffle, y_shuffle

def DataTwoStream(x_total, y_total, val_ratio = 0.2):
    index_split = int(x_total.shape[0]*0.8)
    x_train_a = x_total[:index_split,:,0]
    x_train_a = x_train_a.reshape((x_train_a.shape[0],x_train_a.shape[1],1))
    x_train_b = x_total[:index_split,:,1]
    x_train_b =x_train_b.reshape((x_train_a.shape[0],x_train_a.shape[1],1))
    y_train = y_total[:index_split]
    x_val_a = x_total[index_split:,:,0]
    x_val_a = x_val_a.reshape((x_val_a.shape[0],x_val_a.shape[1],1))
    x_val_b = x_total[index_split:,:,1]
    x_val_b = x_val_b.reshape((x_val_a.shape[0],x_val_a.shape[1],1))
    y_val = y_total[index_split:]
    return x_train_a, x_train_b, y_train, x_val_a, x_val_b, y_val

def GetBinData(class_name):
    file_list = os.listdir(os.path.join(os.getcwd(), "data", "npy", "bin_order"))
    
    for i in range(len(file_list)):
        re_x = class_name + '.*(?<!y\.npy)$'
        re_y = class_name + '(.+?)y.npy'
        mx = re.search(re_x, file_list[i])
        my = re.search(re_y, file_list[i])
        if mx:
            filex_idx = i
        if my:
            filey_idx = i
    
    x_order_addr = os.path.join(os.getcwd(), "data", "npy", "bin_order", file_list[filex_idx])
    y_order_addr = os.path.join(os.getcwd(), "data", "npy", "bin_order", file_list[filey_idx])
    x_shuffle, y_shuffle = ShuffleData(x_order_addr, y_order_addr)
    x_train_a, x_train_b, y_train, x_val_a, x_val_b, y_val = DataTwoStream(x_shuffle, y_shuffle)
    class_name = file_list[i][:-8]
    return x_train_a, x_train_b, y_train, x_val_a, x_val_b, y_val

Since I only want to train the last layer, I freeze all the pretrained weights in the 5-classes model which is called base_model here. (based_model + dense) is called head_model here. 

In [17]:
x_train_a, x_train_b, y_train, x_val_a, x_val_b, y_val = GetBinData('A4')

n_classes = 2
#Initializer=keras.initializers.glorot_normal(seed=None)
Initializer=keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)
#optimizer = RMSprop(lr=0.00001, rho=0.9, epsilon=1e-6)
optimizer = SGD(lr=0.00001)
json_file = open('model_5classes_dense64.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
base_model = model_from_json(loaded_model_json)
base_model.load_weights("471-0.907-5classes-dense64.hdf5")
######################################## New Model architecture ##############################################
x = base_model.output
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
#x = Dense(2, activation='relu', kernel_initializer = Initializer, name='dense_-2')(x)
#x = Dropout(0.2)(x)
#x = BatchNormalization()(x)
predictions = Dense(1, activation = 'softmax', kernel_initializer = Initializer, name='dense_-1')(x)
head_model = Model(input = base_model.input, output = predictions)
for layer in base_model.layers:
    layer.trainable = False
    
head_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
head_model.summary()
##################################### Helpers in callbacks ##############################################
tb = TensorBoard(log_dir=os.path.join('tensorboard', 'logs'))
early_stopper = EarlyStopping(patience=10)
csv_logger = CSVLogger(os.path.join('logs', str(time.time()) + '.log'))
checkpointer = ModelCheckpoint(filepath=os.path.join('checkpoints','{epoch:03d}-{val_acc:.3f}.hdf5'),
                                verbose=1,save_best_only=True)
history = History()
callback = Callback()
print_weights1 = LambdaCallback(on_epoch_end=lambda batch, logs: print('layer-1', head_model.layers[-1].get_weights()))
#########################################################################################################


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 60, 1)        0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 60, 1)        0                                            
__________________________________________________________________________________________________
lstm_21 (LSTM)                  (None, 60, 32)       4352        input_11[0][0]                   
__________________________________________________________________________________________________
lstm_23 (LSTM)                  (None, 60, 32)       4352        input_12[0][0]                   
__________________________________________________________________________________________________
dropout_21

C:\Users\Yue Ma\Anaconda\lib\site-packages\ipykernel\__main__.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [18]:
hist = head_model.fit([x_train_a, x_train_b], y_train,
            batch_size=64, epochs=2000, 
            validation_data=([x_val_a, x_val_b], y_val),
            callbacks=[tb, early_stopper, csv_logger, checkpointer, history, callback, print_weights1])

   
model_json = head_model.to_json()
with open("model_head.json", "w") as json_file:
    json_file.write(model_json)
head_model.save_weights("model_head.h5")
print("Saved model to disk")

Train on 1802 samples, validate on 451 samples
Epoch 1/2000
1802/1802 [==============================] - 6s 3ms/step - loss: 7.0688 - acc: 0.5566 - val_loss: 6.7517 - val_acc: 0.5765

Epoch 00001: val_loss improved from inf to 6.75165, saving model to checkpoints\001-0.576.hdf5
layer-1 [array([[ 0.04330691],
       [-0.04108568]], dtype=float32), array([ 0.], dtype=float32)]
Epoch 2/2000
1802/1802 [==============================] - 1s 676us/step - loss: 7.0688 - acc: 0.5566 - val_loss: 6.7517 - val_acc: 0.5765

Epoch 00002: val_loss did not improve from 6.75165
layer-1 [array([[ 0.04330691],
       [-0.04108568]], dtype=float32), array([ 0.], dtype=float32)]
Epoch 3/2000
1802/1802 [==============================] - 1s 711us/step - loss: 7.0688 - acc: 0.5566 - val_loss: 6.7517 - val_acc: 0.5765

Epoch 00003: val_loss did not improve from 6.75165
layer-1 [array([[ 0.04330691],
       [-0.04108568]], dtype=float32), array([ 0.], dtype=float32)]
Epoch 4/2000
1802/1802 [====================

In [7]:
head_model.layers

In [14]:
y_val

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1,

In [16]:
y_train[100:120]

array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1])